In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import scipy as sp

# Loading and Preprocessing Data via TensorFlow

Usually working with datasets that will not fit in RAM. Can use the **TensorFlow Dataset API**, which will take care of optimizations including...
- Multithreading
- Queuing
- Batching
- Prefetching

The Data API helps bringing in from binary, tensorflowbinary, csv, or SQL files/databases, but can also help in its preprocessing.

Two things we will focus on:
- _TF Transform_ (tf.Transform) - Helps to write a preprocessing function to run in batch mode on the training data such that it can be incorporated into the training model where once it is deployed, it will automatically incorporate new instances.
- _TF Datasets_ (TFDS) - Can download many existing datasets and can use database objects for convenient manipulation.



***
## The Data API

### Introduction

Below is an example that can fit entirely in RAM, but it serves as a starting point.

In [2]:
X = tf.range(10)  # any data tensor
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

**from_tensor_slices()** method takes a Tensor and creates a tf.data.Dataset object with elements that are slices of X defaulting to the first dimension.

In [3]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


### Chaining Transformations

An example of a transformation chain is seen below:

![Chaining TensorFlow Transformations](chaining_transformations_tf_dataset.PNG)



In [4]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


Can even set the drop_remainder flag to True to ensure all the batches have the same size.

These dataset methods create new methods, so they need to be saved to a reference. But their elements can also be mutated with the .map method.

In [5]:
dataset = dataset.map(lambda x: x * 2)
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


**.map()** method is the one that will be called if your dataset needs preprocessing before being fed into the network. 
- map(): each item
- apply(): dataset as a whole

Can also filter the dataset:

In [6]:
X = tf.range(10)  # any data tensor
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset = dataset.map(lambda x: x * 2)
dataset = dataset.filter(lambda x: x < 10)
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)


Looking at just a few items (akin to the Pandas DataFrame .head() method), we can use .take():

In [7]:
for item in dataset.take(3):
     print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


### Shuffling the Data

Gradient descent works best when the dataset is identically distributed. Can use the .shuffle method to accomplish this. Fills a buffer with a specific number of items in the dataset and randomly pulls them out. Set the buffer to be accomodating for the amount of RAM your computer has. 



In [8]:
dataset = tf.data.Dataset.range(10).repeat(3) # 0 to 9, three times
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7, drop_remainder=True)
for item in dataset:
     print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)


Another approach is to split the source data into multiple files and reorder the training database based on the order of the files. 



### Interleaving the Data

Let's load the California housing dataset as practice.

In [9]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("/gv1/users/csmith657/ML Practice Datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [10]:
fetch_housing_data()

In [11]:
housing = load_housing_data()

In [12]:
housing.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [13]:
housing = housing.iloc[:, :-1]

In [14]:
housing.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


The book is saying that we are assuming that we have already split the data into a...

- training set
- validation set
- test set

and then have split those sets into many .csv files. Essentially, that we have created our own effective batches and saved them as separate .csv files. We haven't done that yet, so let's do it now.



In [15]:
housing.shape

(20640, 9)

We can shuffle the DataFrame in-place using Pandas:

In [16]:
housing = housing.sample(frac=1).reset_index(drop=True)

Once it's been shuffled, we can then split the data 60/20/20 between the training set, validation set, and the test sets.

In [17]:
num_rows = len(housing.index)
train_rows = int(np.floor(num_rows*0.6))
valid_rows = int(np.floor(num_rows*0.2))
test_rows = int(np.floor(num_rows*0.2))

housing_train = housing.iloc[0:train_rows, :]
housing_valid = housing.iloc[train_rows:(train_rows+valid_rows), :]
housing_test = housing.iloc[(train_rows+valid_rows):(train_rows+valid_rows+test_rows), :]

housing_train.reset_index(drop=True, inplace=True)
housing_valid.reset_index(drop=True, inplace=True)
housing_test.reset_index(drop=True, inplace=True)

In [18]:
print(housing_train.shape)
print(housing_valid.shape)
print(housing_test.shape)

(12384, 9)
(4128, 9)
(4128, 9)


Now that we have the separations into DataFrames of the separate sets. We can then iterate over the rows and create a multitude of .csv files.

In [19]:
SPLIT_HOUSING_DIR = os.path.join("/gv1/users/csmith657/ML Practice Datasets", "housing_split")

In [20]:
def separate_housing_dat_to_csvs(df, n_csv_files, output_dir, filetitle):
    os.makedirs(output_dir, exist_ok=True)
    all_fp = []
    
    for file in range(n_csv_files):
        full_path = os.path.join(output_dir, f"{filetitle}_{file}.csv")
        df[np.arange(len(df.index))//n_csv_files==file].to_csv(full_path, index=False)
        all_fp.append(full_path)
        
    return all_fp

In [21]:
train_fp = separate_housing_dat_to_csvs(df=housing_train,
                                        n_csv_files=50,
                                        output_dir=SPLIT_HOUSING_DIR,
                                        filetitle="my_train")

In [22]:
valid_fp = separate_housing_dat_to_csvs(df=housing_train,
                                        n_csv_files=25,
                                        output_dir=SPLIT_HOUSING_DIR,
                                        filetitle="my_valid")

In [23]:
test_fp = separate_housing_dat_to_csvs(df=housing_train,
                                       n_csv_files=25,
                                       output_dir=SPLIT_HOUSING_DIR,
                                       filetitle="my_test")

Now we can create a TensorFlow dataset from these filepaths with the list_files() method, which automatically shuffles the filepaths:

In [24]:
train_fp_dataset = tf.data.Dataset.list_files(train_fp, seed=42)

In [25]:
valid_fp_dataset = tf.data.Dataset.list_files(valid_fp, seed=42)

In [26]:
test_fp_dataset = tf.data.Dataset.list_files(test_fp, seed=42)

The interleave() method then will read from some integer number of files at a time and 

In [27]:
n_readers = 5
train_dataset = train_fp_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

In [28]:
valid_dataset = valid_fp_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

In [29]:
test_dataset = test_fp_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

Now, we can look at the first rows of five of the .csv files that were taken when the train_dataset was made:

In [30]:
for line in train_dataset.take(5):
    print(line.numpy())

b'-117.72,34.1,52.0,2867.0,496.0,978.0,513.0,3.1477,291200.0'
b'-118.47,34.06,45.0,3030.0,433.0,916.0,399.0,9.4664,500001.0'
b'-120.72,38.24,32.0,2685.0,543.0,1061.0,492.0,2.5473,101600.0'
b'-118.52,34.19,37.0,1560.0,275.0,763.0,284.0,3.8516,206900.0'
b'-118.43,34.43,5.0,21113.0,4386.0,9842.0,3886.0,4.2037,194600.0'


### Preprocessing the Data

Say we want to scale the data as a form of preprocessing. We can find the mean and standard deviation of the columns and scale them as

In [31]:
n_inputs = 8

X_mean_series = housing.mean(axis=0) 
X_std_series = housing.std(axis=0)

X_mean = np.asarray(X_mean_series.values)
X_std = np.asarray(X_std_series.values)

In [32]:
print(X_mean_series)
print(X_mean)

longitude               -119.569704
latitude                  35.631861
housing_median_age        28.639486
total_rooms             2635.763081
total_bedrooms           537.870553
population              1425.476744
households               499.539680
median_income              3.870671
median_house_value    206855.816909
dtype: float64
[-1.19569704e+02  3.56318614e+01  2.86394864e+01  2.63576308e+03
  5.37870553e+02  1.42547674e+03  4.99539680e+02  3.87067100e+00
  2.06855817e+05]


In [33]:
print(X_std_series)
print(X_std)

longitude                  2.003532
latitude                   2.135952
housing_median_age        12.585558
total_rooms             2181.615252
total_bedrooms           421.385070
population              1132.462122
households               382.329753
median_income              1.899822
median_house_value    115395.615874
dtype: float64
[2.00353172e+00 2.13595240e+00 1.25855576e+01 2.18161525e+03
 4.21385070e+02 1.13246212e+03 3.82329753e+02 1.89982172e+00
 1.15395616e+05]


In [34]:
def preprocess(line):
  defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
  fields = tf.io.decode_csv(line, record_defaults=defs, select_cols=[i for i in range(n_inputs+1)])
  x = tf.stack(fields[:-1])
  y = tf.stack(fields[-1])
  return (x - X_mean[:-1]) / X_std[:-1], y

Walkthrough of the above preprocessing function:
- Assuming X_mean and X_std are the mean and standard deviation of the column we want.
- The decode_csv() method...
 - Takes a line and an array of default values
 - Tells TensorFlow the number of columns and the types of data
 - Above code says all are floats 
- decode_csv() method returns a list of scalar tensors (one per column). They are stacked.
- Columns are then scaled and returned

### Putting it Together



In [35]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=2, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return dataset.batch(batch_size).prefetch(1)

![Multiple .csv Dataset Preprocessing](multiple_csv_data_pipeline_example.PNG)

### Prefetching

Prefetching tells TensorFlow to read the data from disk and process it while a training algorithm is working.

>> If loading an preprocessing are multithreaded, this is where multiple cores will dramatically improve performance, ensuring the GPU is 100% dedicated to training and the CPU is 100% dedicated to getting the data ready for training.

![Why Prefetching is Important for Large Datasets](importance_of_prefetching.PNG)

If the data can fit into RAM:
- Use the TensorFlow dataset cache() method to cache it to RAM
- Do it after preprocessing
- Do it before shuffling, repeating, and batching
- That way, each instance is preprocessed only once, and the data will be shuffled differently @ each epoch

For **other dataset manipulation methods**:
- concatenate()
- zip()
- window()
- reduce()
- shard()
- flat_map()
- padded_batch()

>> For **creating your own datasets**, from_generator() or from_tensors() will be very useful



### Integrating the Dataset Preprocessing into Keras

Now we use the function we just created to bring in our datasets:

In [36]:
train_set = csv_reader_dataset(train_fp)
valid_set = csv_reader_dataset(valid_fp)
test_set = csv_reader_dataset(test_fp)

And we can then build and train a Keras model that will perform on these datasets.



#### Implementing Dataset Processing on Regression Model

##### Defining the Model

Below is the definition of the model that I will train.

In [55]:
# Building a DNN for housing price prediction
def build_housing_reg_model(n_hidden=5,
                            n_neurons=200,
                            learning_rate=0.001,
                            beta_1=0.9,
                            beta_2=0.999,
                            input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden-1):
        model.add(keras.layers.Dense(n_neurons,
                                     activation="elu",
                                     kernel_initializer="he_normal"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.Dense(n_neurons, activation="elu"))
    model.add(keras.layers.Dense(1, activation="elu"))
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

##### Using the TensorBoard to Visualize its Training

To use the TensorBoard, the model needs to output binary files called _event files_, a grouping of which constitutes a record or a _summary_. TensorBoard monitors the log directory and updates its visualization based on the files in there. 

First, we define the root log directory where we will organize the summaries of different runs, organized in their own subdirectories labeled with each run's date and time such that we can compare different runs.

In [56]:
import os
import time

In [57]:
# Define a method to create log directory for the
# runs at various datetimes
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

Just like with the early stopping callback to ensure we aren't overtraining, we can callback to the TensorBoard.

In [58]:
# Define the TensorBoard callback
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

Then, to start the TensorBoard server, we can run calls directly within Jupyter:

In [59]:
# %load_ext tensorboard
# %tensorboard --logdir=./my_logs --port=6006

##### Training the Regression Model

Let's build it and train it.

In [60]:
housing_reg_model = build_housing_reg_model()

In [61]:
history = housing_reg_model.fit(train_set, epochs=100,
                                validation_data=valid_set,
                                callbacks=[tensorboard_cb])

Epoch 1/100
79/79 [==============================] - 3s 32ms/step - loss: 57888729010.2279 - val_loss: 0.0000e+00
Epoch 2/100
79/79 [==============================] - 1s 11ms/step - loss: 57457019144.0105 - val_loss: 55695436902.4000
Epoch 3/100
79/79 [==============================] - 1s 10ms/step - loss: 55639074631.8355 - val_loss: 54383841689.6000
Epoch 4/100
79/79 [==============================] - 1s 10ms/step - loss: 55320470757.0649 - val_loss: 52203655987.2000
Epoch 5/100
79/79 [==============================] - 1s 12ms/step - loss: 52416319820.2426 - val_loss: 49178786406.4000
Epoch 6/100
79/79 [==============================] - 1s 10ms/step - loss: 49669247144.4026 - val_loss: 45579362201.6000
Epoch 7/100
79/79 [==============================] - 1s 10ms/step - loss: 46086110615.2150 - val_loss: 40338434662.4000
Epoch 8/100
79/79 [==============================] - 1s 10ms/step - loss: 40723059966.5742 - val_loss: 35161245081.6000
Epoch 9/100
79/79 [===========================

TODO: save a JSON file of the model parameters in the TensorBoard logfiles.

Based on [this StackOverflow threat](https://datascience.stackexchange.com/questions/19578/why-my-training-and-validation-loss-is-not-changing), we have encountered the dying ReLU problem. 